In [ ]:
from langchain_community.llms import LlamaCpp  # any HF/gguf model

LLM_MODEL_PATH  = "./models/gemma-3-4b-it.Q2_K/gemma-3-4b-it.Q2_K.gguf"

llm = LlamaCpp(
    model_path=  LLM_MODEL_PATH,
    temperature=0.3,
)                # ← returns an LLM object

# print(llm.invoke("Explain LangChain in one tweet"))


In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain

tmpl = PromptTemplate(
    template="You are a helpful tutor. Explain {topic} in simple terms.",
    input_variables=["topic"],
)

chain = LLMChain(prompt=tmpl, llm=llm)
print(chain.invoke({"topic": "vector databases"}))


In [ ]:
from langchain.chains import SequentialChain

research = PromptTemplate.from_template("List 3 key facts about {topic}.")
tweet    = PromptTemplate.from_template("Turn this into a tweet:\n{facts}")

chain1 = LLMChain(llm=llm, prompt=research, output_key="facts")
chain2 = LLMChain(llm=llm, prompt=tweet,    output_key="tweet")

pipeline = SequentialChain(chains=[chain1, chain2],
                           input_variables=["topic"],
                           output_variables=["tweet"])

print(pipeline.invoke({"topic": "LangChain"}))


In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

memory = ConversationBufferMemory()
chat   = ConversationChain(llm=llm, memory=memory)

chat.invoke("Hi!")            # greet
chat.invoke("What did I just say?")  # model sees previous turn

In [ ]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_community.embeddings import SentenceTransformerEmbeddings


loader = DirectoryLoader(
    path="docs",                     # folder name
    glob="**/*.md",                  # match all .md recursively
    show_progress=True,              # optional
)
docs = loader.load()

# 2. Embed + index

embeddings = SentenceTransformerEmbeddings(
    model_name="./models/all-MiniLM-L6-v2",   # local path or HF name
    model_kwargs={"device": "cpu"}            # "cuda" if your GPU is ready
)
vectordb = FAISS.from_documents(docs, embeddings)   # ← now works

# 3. Ask questions
retriever = vectordb.as_retriever( search_k=4)
qa = RetrievalQA.from_chain_type(llm=llm,
                                 retriever=retriever,
                                 chain_type="stuff")  # or “map_reduce”

print(qa.invoke({"query": "What is our password policy?"}))


In [ ]:
from langchain.tools import Tool
import requests, bs4, re

def web_search(q: str) -> str:        # micro-tool
    return requests.get(f"https://duckduckgo.com/html/?q={q}", timeout=10).text[:4000]

search_tool = Tool.from_function(
    name="duck_search",
    description="search the web",
    func=web_search,
)

from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    tools=[search_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

agent.invoke("LangChain interrupt 2025 conference in one sentence")
